In [1]:
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib

import os
import sys

#import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import torchvision
import torchvision.transforms as transforms

matplotlib.rcParams['figure.figsize'] = (20,10)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
#Ikke i brug endnu

cpu


## Kode til at gemme modeltilstande

In [3]:
def SaveState(model):
    x = datetime.datetime.now()
    filename = '../checkpoints/model_state_%s_%.4f.pt' % (x.strftime("%Y%m%d-%H%M"), acc)
    torch.save(model.state_dict(), filename)
    print('Model saved as:\n%s' % os.path.abspath(filename))
    
def LoadState(filename):
    model = torch.load(filename)
    model.eval()
    return model

## Indlæs data
Det antages at dataen ligger i `'../data'`

In [4]:
data_raw = np.load('../data/cullpdb+profile_5926.npy')

## Omform data

In [5]:
data = data_raw.reshape((-1,700,57))

data.shape
x = data[:,:,:22]
y = data[:,:,22:31] # (brug 35 hvis du vil have solvent properties med)
print('Fuldt datasæt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

y_test_unrot = y[5435:5690]
y_validation_unrot = y[5690:5926]

#x = x.reshape(-1,22,700)
#y = y.reshape(-1,9,700)
x = np.rot90(x, axes=(1,2))
y = np.rot90(y, axes=(1,2))

x = np.flip(x, 1)
y = np.flip(y, 1)

print('Fuldt datasæt vendt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

x_train = x[:5430]
y_train = y[:5430]

x_test = x[5435:5690]
y_test = y[5435:5690]

x_validation = x[5690:5926]
y_validation = y[5690:5926]

print('Splittet ud i training og testing:')
print('(Train) X: ', x_train.shape)
print('(Train) Y: ', y_train.shape)
print('(Test)  X: ', x_test.shape)
print('(Test)  Y: ', y_test.shape)
print('(Validation)  X: ', x_validation.shape)
print('(Validation)  Y: ', y_validation.shape)

torch_X_train = torch.from_numpy(x_train).type(torch.FloatTensor)
torch_Y_train = torch.from_numpy(y_train).type(torch.FloatTensor)
torch_X_test  = torch.from_numpy(x_test).type(torch.FloatTensor)
torch_Y_test  = torch.from_numpy(y_test).type(torch.FloatTensor)
torch_X_validation  = torch.from_numpy(x_validation).type(torch.FloatTensor)
torch_Y_validation  = torch.from_numpy(y_validation).type(torch.FloatTensor)

Fuldt datasæt shape:
X:  (5926, 700, 22)
Y:  (5926, 700, 9)
Fuldt datasæt vendt shape:
X:  (5926, 22, 700)
Y:  (5926, 9, 700)
Splittet ud i training og testing:
(Train) X:  (5430, 22, 700)
(Train) Y:  (5430, 9, 700)
(Test)  X:  (255, 22, 700)
(Test)  Y:  (255, 9, 700)
(Validation)  X:  (236, 22, 700)
(Validation)  Y:  (236, 9, 700)


In [175]:
NoSeq = 8
y_validation_labels = np.argmax(y_validation_unrot, axis=2)
y_validation_mask = y_validation_labels != NoSeq

## Sæt data sammen i DataLoader

In [6]:
BATCH_SIZE = 50 # 250 har virket godt før

train = torch.utils.data.TensorDataset(torch_X_train, torch_Y_train)
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=False)

## Definér modellen

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (700, 22)
            nn.Conv1d(
                in_channels=22,            # input height
                out_channels=30,           # n_filters / feature maps
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (300, 22)
            nn.ReLU(),                      # activation
        )
        self.conv2 = nn.Sequential(         # input shape (700, 22)
            nn.Conv1d(
                in_channels=30,            # input height
                out_channels=20,           # n_filters / feature maps
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (300, 22)
            nn.ReLU(),                      # activation
        )
        self.out = nn.Sequential(         # input shape (700, 22)
            nn.Conv1d(
                in_channels=20,            # input height
                out_channels=9,           # n_filters / feature maps
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (300, 22)
            nn.ReLU(),                      # activation
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        output = self.out(x)
        return output, x    # return x for visualization

## Instantiér modellen

In [8]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv1d(22, 30, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv1d(30, 20, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
  )
  (out): Sequential(
    (0): Conv1d(20, 9, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): Softmax()
  )
)


## Hyperparametre

In [9]:
EPOCH = 25               # train the training data n times
LR = 0.005              # learning rate

## Instantiér loss-funktioner og optimizer

In [10]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.BCELoss()  #Binary Cross Entropy Loss
validation_loss_func = nn.BCELoss()
test_loss_func = nn.BCELoss()

In [ ]:
# Til visualisering senere
losses     = []
accuracies = []
steps      = []
steps_cum  = []
epochs     = []

## Funktion til at måle accuracy
Ikke helt smuk - gør pænere senere

In [158]:
def CalculateAccuracyOld(calc_values, real_values):
    relevant = 0
    calc_values = calc_values.detach().numpy()              # Omform til numpy
    calc_values = np.flip(calc_values, 1)                   # Omgør spejlning og 
    calc_values = np.rot90(calc_values, k=-1, axes=(1,2))   # rotation
    calc_values_oh = np.zeros(calc_values.shape)            # Lav et nyt tomt array med samme shape
    for prot_nr, protein in enumerate(calc_values):         # For hvert protein
        for amino_nr, amino in enumerate(protein):          # For hver aminosyre
            feature = np.argmax(amino)                      # Find den højeste sandsynlighed
            if real_values[prot_nr, amino_nr, feature] != 1:
                relevant += 1
            calc_values_oh[prot_nr, amino_nr, feature] = 1  # Sæt den til 1
    differences = abs(calc_values_oh.flatten() - real_values.flatten())
    print(relevant)
    false = int(sum(differences))
    #total = relevant
    total = len(real_values.flatten())
    #print(total_old)
    true  = total-false
    return (true/total)

In [15]:
def CalculateAccuracy(calc_values, real_values):
    NoSeq = 8
    real_labels = np.argmax(real_values, axis=2)
    real_mask = real_labels == NoSeq
    calc_values = calc_values.detach().numpy()              # Omform til numpy
    calc_values = np.flip(calc_values, 1)                   # Omgør spejlning og 
    calc_values = np.rot90(calc_values, k=-1, axes=(1,2))   # rotation
    calc_labels = np.argmax(calc_values, axis=2)
    correct = calc_labels == real_labels
    correct_masked = np.ma.masked_array(correct, real_mask)
    theMean = np.mean(correct_masked)
    return theMean

## Træn modellen

In [13]:
step_cum = -1
for epoch in range(EPOCH):
    print('\nEpoch: ', epoch+1)
    for step, (b_x, b_y) in enumerate(train_loader):
        step_cum += 1
        output = cnn(b_x)[0]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 10 == 0:
            test_output, last_layer = cnn(torch_X_validation)
            vloss = validation_loss_func(test_output, torch_Y_validation)
            acc = CalculateAccuracy(test_output, y_validation_unrot)
            sys.stdout.write('\rStep [%d]:\tLoss: %.5f,\tAccuracy: %.4f%%' % (step, vloss.item(), acc*100))
            #epochs.append(epoch)
            #losses.append(vloss.item())
            #accuracies.append(acc)
            #steps.append(step)
            #steps_cum.append(step_cum)
            
print('\nDone training.')


Epoch:  1
Step [100]:	Loss: 0.09500,	Accuracy: 30.4884%
Epoch:  2
Step [100]:	Loss: 0.09291,	Accuracy: 30.7277%
Epoch:  3
Step [100]:	Loss: 0.09203,	Accuracy: 30.7298%
Epoch:  4
Step [100]:	Loss: 0.09169,	Accuracy: 30.8352%
Epoch:  5
Step [100]:	Loss: 0.09136,	Accuracy: 30.9894%
Epoch:  6
Step [100]:	Loss: 0.09124,	Accuracy: 31.1070%
Epoch:  7
Step [100]:	Loss: 0.09126,	Accuracy: 31.1699%
Epoch:  8
Step [100]:	Loss: 0.09126,	Accuracy: 31.0989%
Epoch:  9
Step [100]:	Loss: 0.09122,	Accuracy: 31.1273%
Epoch:  10
Step [100]:	Loss: 0.09119,	Accuracy: 31.0948%
Epoch:  11
Step [100]:	Loss: 0.09114,	Accuracy: 31.0888%
Epoch:  12
Step [100]:	Loss: 0.09110,	Accuracy: 31.0806%
Epoch:  13
Step [100]:	Loss: 0.09107,	Accuracy: 31.0685%
Epoch:  14
Step [100]:	Loss: 0.09105,	Accuracy: 31.0583%
Epoch:  15
Step [100]:	Loss: 0.09104,	Accuracy: 31.0583%
Epoch:  16
Step [100]:	Loss: 0.09103,	Accuracy: 31.0360%
Epoch:  17
Step [100]:	Loss: 0.09101,	Accuracy: 31.0502%
Epoch:  18
Step [100]:	Loss: 0.09100,	A

## Kørsel på test-sættet

In [14]:
print('Running on the test set:')
test_output, last_layer = cnn(torch_X_test)
vloss = test_loss_func(test_output, torch_Y_test)
#acc = CalculateAccuracy(test_output, y_test_unrot)
acc = CalculateAccuracyNew(test_output, y_test_unrot)
sys.stdout.write('Loss: %.5f,\tAccuracy: %.4f%%' % (vloss.item(), acc*100))

Running on the test set:
Loss: 0.09327,	Accuracy: 29.9978%

In [149]:
SaveState(cnn)

Model saved as:
/home/simonsen/Documents/Uni/bachelor/git/Bachelor19/checkpoints/model_state_20190527-1433_0.9631.pt
